In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from mnist import MNIST
import numpy as np
import pickle

In [4]:
from keras.datasets import mnist    
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train = (x_train/255).astype(float)
# x_test = (x_test/255).astype(float)

2023-10-16 01:33:24.564662: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 01:33:25.212953: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
x_train[10, 10:20, 10:20]

array([[  0,   0,   0,  24, 209, 254, 254, 254, 171,   0],
       [  0,   0,  91, 137, 253, 254, 254, 254, 112,   0],
       [ 40, 214, 250, 254, 254, 254, 254, 254,  34,   0],
       [ 81, 247, 254, 254, 254, 254, 254, 254, 146,   0],
       [  0, 110, 246, 254, 254, 254, 254, 254, 171,   0],
       [  0,   0,  73,  89,  89,  93, 240, 254, 171,   0],
       [  0,   0,   0,   0,   0,   1, 128, 254, 219,  31],
       [  0,   0,   0,   0,   0,   7, 254, 254, 214,  28],
       [  0,   0,   0,   0,   0, 138, 254, 254, 116,   0],
       [  0,   0,   0,   0,  25, 240, 254, 254,  34,   0]], dtype=uint8)

In [4]:
min_per_class = 1e10
for i in range(10):
    min_per_class = min(min_per_class, len(np.where(y_test == i)[0]))

In [5]:
min_per_class

892

In [6]:
data_indx_train, data_indx_test = [], []
min_per_class_train = 5421
min_per_class_test = 892
train_sz, test_sz = 5421, 892
for i in range(10):
    sample_data_train = np.random.choice(np.where(y_train == i)[0], train_sz, replace=False)
    sample_data_test = np.random.choice(np.where(y_test == i)[0], test_sz, replace=False)
    data_indx_train.append(sample_data_train)
    data_indx_test.append(sample_data_test)

data_indx_train = np.array(data_indx_train)
data_indx_test = np.array(data_indx_test)

In [8]:
# y_test[data_indx_test[5]]

In [9]:
data_train = np.zeros((10, train_sz, 28, 28, 3))
data_test = np.zeros((10, test_sz, 28, 28, 3))

for i in range(10):
    for j in range(train_sz):
        data_train[i, j, :, :, :] = np.repeat(x_train[data_indx_train[i, j],:, :, np.newaxis], repeats=3, axis=2)
    for j in range(test_sz):
        data_test[i, j, :, :, :] = np.repeat(x_test[data_indx_test[i, j],:, :, np.newaxis], repeats=3, axis=2)

# data_train = data_train.astype(np.int8)
# data_test = data_test.astype(np.int8)


In [10]:
data_test[0, 10, 10:20, 10:20, 2]

array([[175.,   3.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 220.],
       [ 60.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 220.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 220.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 220.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 220.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 220.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 220.],
       [  4.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  63., 239.],
       [186.,   3.,   0.,   0.,   0.,   0.,   0.,   0., 241., 253.],
       [253.,  62.,   0.,   0.,   0.,   0.,   3., 170., 253., 253.]])

In [11]:
import tensorflow as tf

image_data_train = tf.constant(data_train, tf.uint8)
image_data_test = tf.constant(data_test, tf.uint8)

2023-10-16 01:33:39.127758: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46492 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:01:00.0, compute capability: 8.9


In [12]:
image_data_train[0, 10, 10:20, 10:20, 2].numpy()

array([[  0,   0,  31, 253, 253, 253, 253, 253, 253, 253],
       [  0,   0,   2,  95, 122,  14,  14,  14, 123, 253],
       [  0,   0,   0,   0,   0,   0,   0,   0,   5, 138],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  7,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [177,   7,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=uint8)

In [14]:
# np.any(data_test<0)
tf.reduce_any(image_data_test<0).numpy()
# image_data_train

False

In [15]:
# Save the tensor to a file using tf.io
tf.io.write_file('minst_full_train.txt', tf.io.serialize_tensor(image_data_train))
tf.io.write_file('minst_full_test.txt', tf.io.serialize_tensor(image_data_test))

In [16]:
import tensorflow as tf
serialized_tensor_test = tf.io.read_file('minst_full_test.txt')
image_data_test = tf.io.parse_tensor(serialized_tensor_test, out_type=tf.uint8)

In [18]:
image_data_train[0, 10, 10:20, 10:20, 2].numpy()

array([[  0,   0,  31, 253, 253, 253, 253, 253, 253, 253],
       [  0,   0,   2,  95, 122,  14,  14,  14, 123, 253],
       [  0,   0,   0,   0,   0,   0,   0,   0,   5, 138],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  7,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [177,   7,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=uint8)

<hr>

In [5]:
fileObj1 = open("./mnist_train" + str(train_sz) + ".pkl", 'wb')
pickle.dump(data_train, fileObj1)

In [6]:
fileObj1.close()

In [7]:
fileObj2 = open("./mnist_test" + str(test_sz) + ".pkl", 'wb')
pickle.dump(data_test, fileObj2)
fileObj2.close()

In [9]:
# with open("./mnist_train" + str(train_sz) + ".pkl", 'rb') as f:
#     rand = pickle.load(f)
# np.all(rand == data_train)

True

In [11]:
import tensorflow as tf

with open('mnist_train'+ str(train_sz) +'.pkl', 'rb') as fileObj:
    image_data_train = pickle.load(fileObj)

with open('mnist_test'+ str(test_sz) +'.pkl', 'rb') as fileObj:
    image_data_test = pickle.load(fileObj)

# CHANGE : Save pkl data as tensor
image_data_train = tf.constant(image_data_train, dtype=tf.float32)
image_data_test = tf.constant(image_data_test, tf.float32)


# Save the tensor to a file using tf.io
tf.io.write_file('mnist_train'+ str(train_sz) +'.txt', tf.io.serialize_tensor(image_data_train))

tf.io.write_file('mnist_test'+ str(test_sz) +'.txt', tf.io.serialize_tensor(image_data_test))

2023-07-14 11:01:46.901127: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4392 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:01:00.0, compute capability: 8.9
2023-07-14 11:01:46.901842: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 6 MB memory:  -> device: 1, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:23:00.0, compute capability: 8.9
2023-07-14 11:01:46.902695: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 43468 MB memory:  -> device: 2, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:41:00.0, compute capability: 8.9
2023-07-14 11:01:46.903255: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 43902 MB memory:  -> device: 3, name: NVIDIA 